# Simulate neutrino interactions with LVD and generate histograms

## Configurations
Fiducial mass: 1 kton cintillator

N_p=1.7e+32

N_e=6.3e+32

N_C=7.7e+31

HTE (high threshold energy): 4 MeV (internal tanks)

LTE: 0.8 MeV

Operational time: 30 yrs


In [1]:
# FUNCTIONS

## LVD resolution
def sigma_res(E):
    import numpy as np
    return 0.23*E*np.sqrt(1+0.1*E)/np.sqrt(E)

# Generate random energies acording to Garching flux
def random_gen_garching(f):
    import numpy as np
    
    Emax=100
    
    M=max(f( np.linspace(0,Emax,1000)))
    
    x=Emax*np.random.rand()
    y=M*np.random.rand()
    while y>f(x):
        x=Emax*np.random.rand()
        y=M*np.random.rand()
    return x

In [2]:
# CODE

import numpy as np
import scipy.integrate as integrate
from matplotlib import pyplot as plt
import math

# Detector parameters
N_p=1.7e+32 # number of p
#N_e=6.3e+32 # number of e-
N_C=7.7e+31 # number o 12c
T= 1000*(365*24*3600 )  #operational time in s
HTE=4 # /MeV
LTE=0.8 # /MeV

# Calculate flux parameters from emission spectrum

## Emission parameters: Etot/MeV, E0/MeV, pinching parameter
par0_e=[5e+52*624151, 8, 2.3]
par0_ebar=[5e+52*624151, 10, 2.3]
par0_x=[5e+52*624151, 12, 2.3]

from snr import snr_yuksel
J1,J2,J3=snr_yuksel(0,'integrals','avg')

### Thermal emission spectra
A=(2+2.3)/(1+2.3)
k=J3*J1/J2**2
a=(2-k*A)/(k*A-1)

## Flux parameters: Phi_tot/cm^-2s^-1, E0/MeV, a
par_e=[0,0,a]
par_ebar=[0,0,a]
par_x=[0,0,a]

par_e[0]=J1*par0_e[0]/par0_e[1]
par_ebar[0]=J1*par0_ebar[0]/par0_ebar[1]
par_x[0]= 2  *  J1*par0_x[0]/par0_x[1] # Sum of v_mu + v_tau

par_e[1]=par0_e[1]*J2/J1
par_ebar[1]=par0_ebar[1]*J2/J1
par_x[1]=par0_x[1]*J2/J1

In [3]:
# Average number of interactions

## Fluxes on earth after emission and MSW effect
from spectra import spectrum_garching
f0e= lambda E: spectrum_garching(E,par_e[0]*par_e[1],par_e[1],par_e[2])
f0ebar=lambda E: spectrum_garching(E,par_ebar[0]*par_ebar[1],par_ebar[1],par_ebar[2])
f0x=lambda E: spectrum_garching(E,par_x[0]*par_x[1],par_x[1],par_x[2])

from mix import mix_ad
fe_nh=mix_ad(f0e,f0ebar,f0x,'nu_e','nh')
fe_ih=mix_ad(f0e,f0ebar,f0x,'nu_e','ih')
febar_nh=mix_ad(f0e,f0ebar,f0x,'nu_ebar','nh')
febar_ih=mix_ad(f0e,f0ebar,f0x,'nu_ebar','ih')
fx_nh=mix_ad(f0e,f0ebar,f0x,'nu_x','nh')
fx_ih=mix_ad(f0e,f0ebar,f0x,'nu_x','ih')

## NH

flux_IBD_nh=lambda E: febar_nh(E)

flux_NCnu_nh=lambda E: fe_nh(E) + fx_nh(E)

flux_NCnubar_nh= lambda E: febar_nh(E) + fx_nh(E)

## IH

flux_IBD_ih=lambda E: febar_ih(E)

flux_NCnu_ih=lambda E: fe_ih(E) + fx_ih(E)

flux_NCnubar_ih= lambda E: febar_ih(E) + fx_ih(E)

In [4]:
# Cross sections for IBD and NC scattering on 12C

## NC: vx + C -> vx+ *C for neutrinos
from cross_sections import cs_NC_C_nu
## NC: vx + C -> vx+ *C for anti-neutrinos
from cross_sections import cs_NC_C_nubar

## IBD
from cross_sections import cs_ibd

In [5]:
# NH

## IBD
Navg_IBD_nh,_=integrate.quad(lambda E: N_p*T*flux_IBD_nh(E)*cs_ibd(E),0,np.Inf)

## NC nu (nu_e,nu_x)
Navg_NC_nu_nh,_=integrate.quad(lambda E: N_C*T* flux_NCnu_nh(E) *cs_NC_C_nu(E),0,np.Inf)

## NC nubar (nu_ebar,nu_xbar)
Navg_NC_nubar_nh,_=integrate.quad(lambda E: N_C*T* flux_NCnubar_nh(E)*cs_NC_C_nubar(E),0,np.Inf)

# IH

## IBD
Navg_IBD_ih,_=integrate.quad(lambda E: N_p*T*flux_IBD_ih(E)*cs_ibd(E),0,np.Inf)

## NC nu (nu_e,nu_x)
Navg_NC_nu_ih,_=integrate.quad(lambda E: N_C*T* flux_NCnu_ih(E) *cs_NC_C_nu(E),0,np.Inf)

## NC nubar (nu_ebar,nu_xbar)
Navg_NC_nubar_ih,_=integrate.quad(lambda E: N_C*T* flux_NCnubar_ih(E)*cs_NC_C_nubar(E),0,np.Inf)

In [10]:
Navg_NC_nubar_ih

1.3634726553508345

In [11]:
# ITERACTION OVER RUNS
runs=100

Nev_IBD_nh=np.zeros(runs)
Nev_IBD_ih=np.zeros(runs)
Nev_NC_nh=np.zeros(runs)
Nev_NC_ih=np.zeros(runs)

for run in range(runs):
    
    # Sample k events from average using poisson distribution
    
    ## NH
    
    k_IBD_nh=np.random.poisson(Navg_IBD_nh)
    
    k_NC_nh=np.random.poisson(Navg_NC_nu_nh) + np.random.poisson(Navg_NC_nubar_nh)
    
    ## IH
    
    k_IBD_ih=np.random.poisson(Navg_IBD_ih)
    
    k_NC_ih=np.random.poisson(Navg_NC_nu_ih) + np.random.poisson(Navg_NC_nubar_ih)
    
    
    ## Detected energies (photon)
    Ed_NC_nh=np.array([])
    Ed_NC_ih=np.array([])
    Ed_IBD_nh=np.array([])
    Ed_IBD_ih=np.array([])
    
    for i in range(k_NC_nh):
        r=random_gen_garching(lambda E: flux_NCnu_nh(E)+flux_NCnubar_nh(E))
        if r>15.11:
            s=np.random.normal(15.11,sigma_res(15.11))
            Ed_NC_nh=np.concatenate( (Ed_NC_nh, [s]) )
    
    for i in range(k_NC_ih):
        r=random_gen_garching(lambda E: flux_NCnu_ih(E)+flux_NCnubar_ih(E))
        if r>15.11:
            s=np.random.normal(15.11,sigma_res(15.11))
            Ed_NC_ih=np.concatenate( (Ed_NC_ih,[s] ) )

    for i in range(k_IBD_nh):
        r=random_gen_garching(flux_IBD_nh) -0.8 # Egamma = Ev-0.8
        s=np.random.normal(r,sigma_res(r))
        if s>HTE:
            Ed_IBD_nh=np.concatenate( (Ed_IBD_nh,[s]) )

    for i in range(k_IBD_ih):
        r=random_gen_garching(flux_IBD_ih) -0.8 # Egamma = Ev-0.8
        s=np.random.normal(r,sigma_res(r))
        if s>HTE:
            Ed_IBD_ih=np.concatenate( (Ed_IBD_ih,[s]) )
    
    #  Number of events
    Nev_NC_nh[run]=len(Ed_NC_nh)
    Nev_NC_ih[run]=len(Ed_NC_ih)
    
    Nev_IBD_nh[run]=len(Ed_IBD_nh)
    Nev_IBD_ih[run]=len(Ed_IBD_ih)


C:\Users\rafae\AppData\Local\Temp/ipykernel_11364/1094727899.py:6: RuntimeWarning: invalid value encountered in sqrt
  return 0.23*E*np.sqrt(1+0.1*E)/np.sqrt(E)


In [26]:
Nev_NC_nh

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
# Average number of events and std

## NH

avg_NC_nh=np.mean(Nev_NC_nh)
std_NC_nh=np.std(Nev_NC_nh)

avg_IBD_nh=np.mean(Nev_IBD_nh)
std_IBD_nh=np.std(Nev_IBD_nh)

## IH

avg_NC_ih=np.mean(Nev_NC_ih)
std_NC_ih=np.std(Nev_NC_ih)

avg_IBD_ih=np.mean(Nev_IBD_ih)
std_IBD_ih=np.std(Nev_IBD_ih)

201.07

In [ ]:
# Histograms

# NC events seen all at E=15.11 MeV

# "Sum" of all runs
E_runs=[]
for i in range(runs):
    gauss_dist=np.random.normal(15.11,sigma(15.11),int(Nev_NC[i]))
    E_runs=np.concatenate( (E_runs,gauss_dist) )


In [ ]:
def f2(f):
    fun=lambda x: f(x)+2
    return fun

In [ ]:
y=lambda z: 2*z

f2(y)(1)